<img src="https://www.docker.com/wp-content/uploads/2022/03/horizontal-logo-monochromatic-white.png" width="400" height="200" align="left"> 

## Check images

In [5]:
%%bash
docker images

REPOSITORY          TAG                   IMAGE ID       CREATED         SIZE
micapipe            v0.1.5                88bc128c4d9e   2 days ago      36.7GB
my_fastsurfer       cpu                   6167e77f6578   6 weeks ago     5.49GB
deepmi/fastsurfer   surfonly-cpu-v2.0.0   a1373a0fbbe1   7 weeks ago     1.59GB
deepmi/fastsurfer   cpu-v2.0.0            28f6bd92ba08   7 weeks ago     5.49GB
micalab/micapipe    v0.1.4                1a581843a33a   3 months ago    36.7GB
ubuntu              20.04                 680e5dfb52c7   4 months ago    72.8MB
ubuntu              16.04                 b6f507652425   18 months ago   135MB
ubuntu              latest                f643c72bc252   2 years ago     72.9MB
ubuntu              bionic-20201119       2c047404e52d   2 years ago     63.3MB


## Remove images
```bash
docker rmi -f <docker-image>
```

## Get inside the container
```bash
docker run -it --entrypoint /bin/bash micapipe:v0.1.5
```

## Build container
To keep all the functions executable you MUST be sure to change the permissions to others before building the container: `chmod aug+xX -R *`
From the main repository where the `Dockerfile` is locate, particularry in the case of `micapipe`, I run, `-t` is for tag:
```bash
docker build . -t micapipe:v0.2.0_x
```

## Taggin the version via docker
If the built was successful the next step is to tag the containe via docker:
```bash
docker tag micapipe:v0.2.0_x micalab/micapipe:v0.2.0
```

## Deploy container to `dockerhub`
The final step is to release the new contianer to the docker hub, this step will require the user credentials e.g. `micalab`.
```bash
docker login --username micalab

docker push micalab/micapipe:v0.2.0
```

## Build a Singularity from the local `docker`
```bash
singularity build micapipe_v0.2.0.sif docker-daemon://micapipe:v0.2.0_x
```

# Sigularity container <img src="https://sylabs.io/wp-content/uploads/2022/03/singularity-logo-round.svg" width="100" height="100" align="left">

### Build `fastSurfer v2.0.4` singularity container 

1. Download the docker version

```bash
docker pull deepmi/fastsurfer:cpu-v2.0.4
```

2. Build the Singularity container

```bash

# set the singularity cache and tmp directories
export SINGULARITY_CACHEDIR=/host/yeatman/local_raid/rcruces/data/images/cache_dir
export SINGULARITY_TMPDIR=/host/yeatman/local_raid/rcruces/data/images/temp_dir

# build the container from the docker image
singularity build fastsurfer-cpu-v2.0.4.sif docker-daemon://deepmi/fastsurfer:cpu-v2.0.4
```



In [10]:
%%bash


# Github actions 
## Create a container after each push on main (master)

# CirlceCI Continuous integration workflow
<img src="https://global-uploads.webflow.com/622659e406588ec12d1336d5/624789be8ca70284b471a6bd_circleci-ar21.png" width="100" height="100" align="left">

# docker `micapipe:v0.2.0_beta`
### Code to run all modules for one subject with the container
## MICs 3T
```bash
# ALL steps
bids=/data/mica3/BIDS_CI/rawdata
out=/data/mica3/BIDS_CI/dev_docker1
fs_dir=/data_/mica3/BIDS_CI/license_fc.txt
tmp=/tmp
docker run -ti --rm --userns=host \
	-v ${bids}:/bids \
    -v ${out}:/out \
    -v ${tmp}:/tmp \
    -v ${fs_dir}:/opt/licence.txt \
    micapipe:v0.2.0_beta \
    -bids /bids -out /out -fs_licence /opt/licence.txt -threads 15 \
    -sub HC010 -ses 01 -proc_structural -proc_surf -post_structural -atlas economo,aparc -proc_dwi -GD -proc_flair -proc_func -MPC -mpc_acq T1map -regSynth
```

```bash
# NO running -proc_surf Run all with -surf_dir
surf_dir=/data_/mica3/BIDS_CI/dev_mica/fastsurfer/sub-HC010_ses-01
docker run -ti --rm --userns=host \
    -v ${bids}:/bids \
    -v ${out}:/out \
    -v ${tmp}:/tmp \
    -v ${surf_dir}:/surf_dir \
    -v ${fs_dir}:/opt/licence.txt \
    micapipe:v0.2.0_beta \
-bids /bids -out /out -fs_licence /opt/licence.txt -threads 15 -sub HC010 -ses 01 -proc_surf -surf_dir /surf_dir \
-post_structural -atlas economo,aparc -proc_dwi -GD -proc_flair -proc_func -MPC -mpc_acq T1map -regSynth

```


## PNI 7T

# singularity `micapipe:v0.2.0_beta`
### Code to run all modules for one subject with the container
## MICs 3T
```bash
# Singularity container
container=/host/yeatman/local_raid/rcruces/git_here/micapipe/micapipe_v0.2.0.sif
singularity exec --nv ${container} -h
singularity exec --nv \
    -B "${bids}":/bids \
    -B "${out}":/output \
    -B "${tmp}":/tmp \
    -B "${fs_dir}":/fs
    "${container}" micapipe \
    -bids /bids -out /output -tmpDir /tmp \
    -fs_licence /fs/license.txt -threads 15 \
    -sub HC010 -ses 01 -proc_structural -proc_surf
```

# `dockerfile` for `freesurfer 7.4.0`
```bash
ENV FREESURFER_HOME="/opt/freesurfer-7.4.0" \
    PATH="/opt/freesurfer-7.4.0/bin:$PATH"
RUN apt-get update -qq \
    && apt-get install -y -q --no-install-recommends \
           bc \
           libgomp1 \
           libxmu6 \
           libxt6 \
           perl \
           tcsh \
    && apt-get clean \
    && rm -rf /var/lib/apt/lists/* \
    && echo "Downloading FreeSurfer ..." \
    && mkdir -p /opt/freesurfer-7.4.0 \
    && curl -fsSL --retry 5 ftp://surfer.nmr.mgh.harvard.edu/pub/dist/freesurfer/7.4.0/freesurfer-linux-ubuntu18_amd64-7.4.0.tar.gz \
    | tar -xz -C /opt/freesurfer-7.4.0 --strip-components 1 \
         --exclude='freesurfer/average/mult-comp-cor' \
         --exclude='freesurfer/lib/cuda' \
         --exclude='freesurfer/lib/qt' \
         --exclude='freesurfer/subjects/V1_average' \
         --exclude='freesurfer/subjects/bert' \
         --exclude='freesurfer/subjects/cvs_avg35' \
         --exclude='freesurfer/subjects/cvs_avg35_inMNI152' \
         --exclude='freesurfer/subjects/fsaverage3' \
         --exclude='freesurfer/subjects/fsaverage4' \
         --exclude='freesurfer/subjects/fsaverage5' \
         --exclude='freesurfer/subjects/fsaverage6' \
         --exclude='freesurfer/subjects/fsaverage_sym' \
         --exclude='freesurfer/trctrain' \
    && sed -i '$isource "/opt/freesurfer-7.4.0/SetUpFreeSurfer.sh"' "$ND_ENTRYPOINT"
```